In [ ]:
%config IPCompleter.use_jedi = False
%pdb off
%load_ext autoreload
%autoreload 3
# %matplotlib inline
%matplotlib qt5
# import mne
# mne.viz.set_browser_backend("qt")  # or "matplotlib"
# mne.set_config("MNE_BROWSER_BACKEND", "qt")  # or "matplotlib"
%gui qt
import numpy as np

from neuropy.utils.matplotlib_helpers import matplotlib_configuration_update

_restore_previous_matplotlib_settings_callback = matplotlib_configuration_update(is_interactive=True, backend='Qt5Agg')

import pyphoplacecellanalysis
# import pyphoplacecellanalysis.External.python_peak_promience2d.peak_promience2d
from pyphoplacecellanalysis.External.peak_prominence2d import getProminence

In [ ]:
#------------------A toy example------------------
xx=np.linspace(-10,10,100)
yy=np.linspace(-10,10,100)

XX,YY=np.meshgrid(xx,yy)
slab=np.zeros(XX.shape)

# add 3 peaks
slab+=5*np.exp(-XX**2/1**2 - YY**2/1**2)
slab+=8*np.exp(-(XX-3)**2/2**2 - YY**2/2**2)
slab+=10*np.exp(-(XX+4)**2/2**2 - YY**2/2**2)

step=0.2
zmax=slab.max()
peaks, idmap, promap, parentmap = getProminence(slab, step, ybin_centers=yy, xbin_centers=xx, min_area=None, include_edge=True)


In [ ]:

#-------------------Plot------------------------
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

from neuropy.utils.matplotlib_helpers import matplotlib_configuration_update

_restore_previous_matplotlib_settings_callback = matplotlib_configuration_update(is_interactive=True, backend='Qt5Agg')

figure=plt.figure(figsize=(12,10),dpi=100)

ax1=figure.add_subplot(2,2,1)

ax1.contourf(XX,YY,slab,levels=np.arange(0,zmax,1))
ax1.set_xlabel('X')
ax1.set_ylabel('Y')
ax1.set_title('Top view, col contours as dashed lines')

for kk,vv in peaks.items():
    print (kk)
    cols=vv['contour']
    ax1.plot(cols.vertices[:,0],cols.vertices[:,1],'k:')

line=slab[slab.shape[0]//2]
ax2=figure.add_subplot(2,2,2)
ax2.plot(xx,line,'b-')
ax2.set_xlabel('X')
ax2.set_ylabel('Z')
ax2.set_title('Cross section through y=0')

for kk,vv in peaks.items():
    xii,yii=vv['center']
    z2ii=vv['height']
    pro=vv['prominence']
    z1ii=z2ii-pro
    ax2.plot([xii,xii], [z1ii,z2ii],'k:')
    ax2.text(xii,z2ii,'p%d, parent = %d' %(kk,vv['parent']),
            horizontalalignment='center',
            verticalalignment='bottom')

ax3=figure.add_subplot(2,2,3,projection='3d')
ax3.plot_surface(XX,YY,slab,rstride=4,cstride=4,cmap='viridis',alpha=0.8)

for kk,vv in peaks.items():
    xii,yii=vv['center']
    z2ii=vv['height']
    pro=vv['prominence']
    z1ii=z2ii-pro
    ax3.plot([xii,xii],[yii,yii],[z1ii,z2ii], color='r', linewidth=2)

ax4=figure.add_subplot(2,2,4)
cs=ax4.imshow(promap,origin='lower',interpolation='nearest',
        extent=[-10,10,-10,10])
ax4.set_xlabel('X')
ax4.set_ylabel('Y')
ax4.set_title('Top view, prominences at peaks')
plt.colorbar(cs,ax=ax4)

plt.show(block=False)

from pprint import pprint
pprint(peaks)